In [21]:
import random
#from operator import attrgetter

kolory=["♠️","♥️","♣️","♦️"]
wartości=["A","K","Q","W","10","9","8","7","6","5","4","3","2"]
pozycje=["N","E","S","W"]

#awersy
piki=['🂡', '🂢', '🂣', '🂤', '🂥', '🂦', '🂧', '🂨', '🂩', '🂪', '🂫', '🂭', '🂮']
kiery=['🂱', '🂲', '🂳', '🂴', '🂵', '🂶', '🂷', '🂸', '🂹', '🂺', '🂻', '🂽', '🂾']
karo=['🃁', '🃂', '🃃', '🃄', '🃅', '🃆', '🃇', '🃈', '🃉', '🃊', '🃋', '🃍', '🃎']
trefle=['🃑', '🃒', '🃓', '🃔', '🃕', '🃖', '🃗', '🃘', '🃙', '🃚', '🃛', '🃝', '🃞']
jokery=['🂿', '🃏']

class Karta():
    '''definicja karty do gry''' 
    def __init__(self, wartość: str = '', kolor: str = ''):
        self.kolor = kolor.strip()
        self.wartość = wartość.strip()

    def __str__(self)->str:
        return f'{self.get_label()}'
    
    def __repr__(self):
        return f'{self.__class__.__name__}(wartość={self.wartość!r}, kolor={self.kolor!r})'
    
    def get_label(self)->str:
            return f'{'?' if self.wartość == '' and self.kolor == '' else self.wartość+self.kolor}'
    
class CardsCollection(list[Karta]):
    '''definicja stosu kart'''
    def __init__(self, *args: list[Karta]|None):
        '''utworzenie stosu kart'''
        super().__init__(*args)
    
    def __str__(self):
        return f'{self.get_labels(self)}'
    
    def __repr__(self):
        return f'{self.__class__.__name__}({[karta.__repr__() for karta in self]})'
    
    def w_stosie(self:CardsCollection)->list[str]:
        nazwy_kart:list[str]=[]
        for karta in self:
            nazwy_kart.append(karta.get_label())
        return(nazwy_kart)
    
    def liczba_kart(self)->int:
        '''liczba kart w stosie'''
        return len(self)
    
    @staticmethod
    def get_labels(stack: CardsCollection|list[Karta])->list[str]:
        '''lista nazw kart w stosie'''
        nazwy_kart:list[str]=[]
        for karta in stack:
            nazwy_kart.append(karta.get_label())
        return(nazwy_kart)
 
class Talia(CardsCollection):
    '''definicja talii kart do gry'''
    
    def __init__(self, *args: list[Karta]|None):
        '''utworzenie talii kart'''
        super().__init__(*args)
        if len(self) == 0: # jeśli nie podano kart, to tworzymy standardową talię 52 kart
            for kolor in kolory:
                for wartość in wartości:
                    self.append(Karta(wartość,kolor))
    
    def __str__(self):
        return f'{self.get_labels(self)}'
    
    def __repr__(self):
        return f'{self.__class__.__name__}({[karta.__repr__() for karta in self]})'
    
    def shuffle(self):
        '''tasowanie talii'''
        random.shuffle(self) # używa algorytmu Fisher-Yates

    def tasuj_fisher_yates(self): # algorytm Fisher-Yates
        '''Fisher-Yates shuffle - algorytm tasowania O(n)'''
        for i in range(len(self) - 1, 0, -1):   # Iteracja od ostatniego elementu do pierwszego
            j = random.randint(0, i)            # Losowy indeks od 0 do bieżącego indeksu (i)
            self[i], self[j] = self[j], self[i] # Zamiana elementów (swap)

    def get_random_card(self)->Karta|None:
        '''losowanie karty z talii'''
        random_card: Karta|None = None
        if len(self) > 0:
            random_card = random.choice(self)
            self.remove(random_card)
        return(random_card)
    
    def get_first_card(self)->Karta|None:
        '''pobranie pierwszej karty z talii'''
        first_card: Karta|None = None
        if len(self) > 0:
            first_card = self.pop(0)
        return(first_card)
    
class Osoba():
    '''definicja osoby'''

    def __init__(self, imię: str=''):
        self.imię = imię
    
    def __str__(self):
        return f'{self.imię}'
    
    def __repr__(self):
        return f'{self.__class__.__name__}(imię={self.imię!r})'
    
class Gracz(CardsCollection, Osoba):
    '''definicja grającego'''

    def __init__(self, *args: list[Karta]|None, imię: str = '', pozycja: str = ''):
        super().__init__(*args)  
        self.imię = imię
        self.pozycja = pozycja
    
    def __str__(self):
        s1 = "Brak imienia" if self.imię == "" else self.imię
        s2 = "Brak pozycji" if self.pozycja  == "" else self.pozycja
        return (f'{s1}, {s2} - {self.hand()}')
    
    def __repr__(self):
        return f'{self.__class__.__name__}(imię={self.imię!r}, pozycja={self.pozycja!r}, hand={self.hand()})'
    
    def store_card(self, karta: Karta|None):
        if karta is not None:
            self.append(karta)
        
    def hand(self)->list[str]:
        return(self.get_labels(self))
    
    def hand_sorted(self)->list[str]:
        posortowane:list[Karta]=sorted(
            self,
            key=lambda karta:(kolory.index(karta.kolor), wartości.index(karta.wartość)),
            reverse=False
        )
        return(self.get_labels(posortowane))
    
class Players(list[Gracz]):
    '''definicja grupy grających'''

    def __init__(self, *args: list[Gracz]|None, deck: Talia|None = None):
        super().__init__(*args)
        self.deck:Talia = deck if deck is not None else Talia()

    def __str__(self):
        return f'{[str(gracz) for gracz in self]}, deck={self.deck.__str__()}'
    
    def __repr__(self):
        return f'{self.__class__.__name__}({super().__repr__()}, deck={self.deck.__repr__()})'
    
    def deal(self): #rozdanie 13 kart każdemu z grających
        for _ in range(13):
            for gracz in self:
                gracz.store_card(self.deck.get_first_card())

In [22]:
# program testujący działanie klas i metod
if __name__ == "__main__" :
    # obsadzenie stołu i rozdanie kart
    the_table = Players(
        [Gracz(imię="Piotr",pozycja=pozycje[0]), 
        Gracz(imię="Agata", pozycja=pozycje[1]), 
        Gracz(imię="Kasia",pozycja=pozycje[2])], 
        deck=Talia())
    the_table.append(Gracz(imię="Karol", pozycja=pozycje[3])) # dodanie kolejnego gracza do stołu

    print(f'stół: {the_table}\n')

    the_table.deck.shuffle() # tasowanie talii
    the_table.deal() # rozdanie kart

    # ręce i talia po rozdaniu
    for gracz in the_table: # rozdane karty każdego gracza
        print(f'rozdane: {gracz}{'\n' if gracz == the_table[-1] else ''}')

    for gracz in the_table: # posortowane karty każdego gracza
        print(f'posortowane: {gracz}{'\n' if gracz == the_table[-1] else ''}')

    print(f'talia po rozdaniu: {the_table.deck}, pozostało: {the_table.deck.liczba_kart()}\n')  

stół: ['Piotr, N - []', 'Agata, E - []', 'Kasia, S - []', 'Karol, W - []'], deck=['A♠️', 'K♠️', 'Q♠️', 'W♠️', '10♠️', '9♠️', '8♠️', '7♠️', '6♠️', '5♠️', '4♠️', '3♠️', '2♠️', 'A♥️', 'K♥️', 'Q♥️', 'W♥️', '10♥️', '9♥️', '8♥️', '7♥️', '6♥️', '5♥️', '4♥️', '3♥️', '2♥️', 'A♣️', 'K♣️', 'Q♣️', 'W♣️', '10♣️', '9♣️', '8♣️', '7♣️', '6♣️', '5♣️', '4♣️', '3♣️', '2♣️', 'A♦️', 'K♦️', 'Q♦️', 'W♦️', '10♦️', '9♦️', '8♦️', '7♦️', '6♦️', '5♦️', '4♦️', '3♦️', '2♦️']

rozdane: Piotr, N - ['3♠️', '4♦️', 'Q♣️', '2♥️', 'Q♥️', 'W♠️', '8♥️', '6♠️', '5♦️', '6♦️', '10♣️', '2♦️', '4♥️']
rozdane: Agata, E - ['A♠️', 'A♣️', '7♠️', 'A♦️', 'K♥️', '9♣️', '2♠️', '4♣️', '2♣️', 'W♥️', '5♥️', '7♥️', '10♥️']
rozdane: Kasia, S - ['Q♠️', '6♣️', '4♠️', 'K♦️', '8♣️', 'K♠️', '9♦️', '3♦️', '5♠️', 'K♣️', '7♣️', '3♣️', '8♠️']
rozdane: Karol, W - ['9♥️', '10♠️', 'A♥️', 'W♣️', '8♦️', '6♥️', 'W♦️', '9♠️', '7♦️', '5♣️', '10♦️', '3♥️', 'Q♦️']

posortowane: Piotr, N - ['3♠️', '4♦️', 'Q♣️', '2♥️', 'Q♥️', 'W♠️', '8♥️', '6♠️', '5♦️', '6♦️', '